###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
#import io
import pickle
from collections import deque
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'

# Decennial 2020 Redistricting Data 

In [4]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']

In [5]:
GNRC = ['111', #Macon
       '161', #Stewart
       '125', #Montgomery
       '083', #Houston
       '085', #Humphreys
       '043', #Dickson
       '021', #Cheatham
       '147', #Robertson
       '165', #Sumner
       '037', #Davidson
       '189', #Wilson
       '169', #Trousdale
       '187', #Williamson
       '149', #Rutherford
       '119'] #Maury
KY = ['003', #Allen
      '213'] #Simpson
qlaces = ['1600000US4700200', #Adams city, Tennessee: Robertson
          '1600000US4702180', #Ashland City town, Tennessee: Cheatham
          '1600000US4704620', #Belle Meade city, Tennessee: Davidson
          '1600000US4705140', #Berry Hill city, Tennessee: Davidson
          '1600000US4708280', #Brentwood city, Tennessee: Williamson
          '1600000US4709880', #Burns town, Tennessee: Dickson
          '1600000US4711980', #Cedar Hill city, Tennessee: Robertson
          '1600000US4713080', #Charlotte town, Tennessee: Dickson
          '1600000US4715160', #Clarksville city, Tennessee: Montgomery
          '1600000US4716540', #Columbia city, Tennessee: Maury
          '1600000US4716980', #Coopertown town, Tennessee: Robertson
          '1600000US4718420', #Cross Plains city, Tennessee: Robertson
          '1600000US4718820', #Cumberland City town, Tennessee: Stewart
          '1600000US4720620', #Dickson city, Tennessee: Dickson
          '1600000US4721400', #Dover city, Tennessee: Stewart
          '1600000US4722360', #Eagleville city, Tennessee: Rutherford
          '1600000US4724320', #Erin city, Tennessee: Houston
          '1600000US4725440', #Fairview city, Tennessee: Williamson
          '1600000US4727020', #Forest Hills city, Tennessee: Davidson
          '1600000US4727740', #Franklin city, Tennessee: Williamson
          '1600000US4728540', #Gallatin city, Tennessee: Sumner
          '1600000US4729920', #Goodlettsville city, Tennessee: Davidson/Sumner
          '1600000US4730960', #Greenbrier town, Tennessee: Robertson
          '1600000US4733280', #Hendersonville city, Tennessee: Sumner
          '1600000US4739660', #Kingston Springs town, Tennessee: Cheatham
          '1600000US4741200', #La Vergne city, Tennessee: Rutherford
          '1600000US4740160', #Lafayette city, Tennessee: Macon
          '1600000US4741520', #Lebanon city, Tennessee: Wilson
          '1600000US4744840', #McEwen city, Tennessee: Humphreys
          '1600000US4748980', #Millersville city, Tennessee: Robertson/Sumner
          '1600000US4749460', #Mitchellville city, Tennessee: Sumner
          '1600000US4750780', #Mount Juliet city, Tennessee: Wilson
          '1600000US4751080', #Mount Pleasant city, Tennessee: Maury
          '1600000US4751560', #Murfreesboro city, Tennessee: Rutherford
          '1600000US4752006', #Nashville-Davidson metropolitan government (balance): Davidson
          '1600000US4752820', #New Johnsonville city, Tennessee: Humphreys
          '1600000US4753460', #Nolensville town, Tennessee: Williamson
          '1600000US4754780', #Oak Hill city, Tennessee: Davidson
          '1600000US4757480', #Pegram town, Tennessee: Cheatham
          '1600000US4759560', #Pleasant View city, Tennessee: Cheatham
          '1600000US4760280', #Portland city, Tennessee: Robertson/Sumner
          '1600000US4763140', #Ridgetop city, Tennessee: Davidson/Robertson
          '1600000US4769080', #Slayden town, Tennessee: Dickson
          '1600000US4769420', #Smyrna town, Tennessee: Rutherford
          '1600000US4770580', #Spring Hill city, Tennessee: Maury/Williamson
          '1600000US4770500', #Springfield city, Tennessee: Robertson
          '1600000US4773460', #Tennessee Ridge town, Tennessee: Houston/Stewart
          '1600000US4773900', #Thompson's Station town, Tennessee: Williamson
          '1600000US4776860', #Vanleer town, Tennessee: Dickson
          '1600000US4778320', #Watertown city, Tennessee: Wilson
          '1600000US4778560', #Waverly city, Tennessee: Humphreys
          '1600000US4779420', #Westmoreland town, Tennessee: Sumner
          '1600000US4779980', #White Bluff town, Tennessee: Dickson
          '1600000US4780200', #White House city, Tennessee: Robertson/Sumner
          '1600000US2128918', #Franklin city, Kentucky: 
          '1600000US2169114'] #Scottsville city, Kentucky


In [6]:
dataguide = pd.read_csv('../../Data Guides/DATA GUIDE PL2020.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [7]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]

In [8]:
#url string and list parameters for column head and tail
url_str= 'https://api.census.gov/data/2020/dec/pl?key='+api_key
head1 = 'NAME' 
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'

In [9]:
dataguide = dg1
var_list = list(dataguide['PL Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in state of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = pd.concat([df, kycos], axis = 0)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = pd.concat([df, places], axis = 0)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = pd.concat([df, places], axis = 0)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, state], axis = 0)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [10]:
data = savename

In [11]:
data = data.set_index(['NAME', 'GEO_ID'])
data.head()

,,pop,raceeth_total_series,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other,StateFIPS,GeoFIPS
NAME,GEO_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Montgomery County, Tennessee",0500000US47125,220069,220069,137775,44569,1154,5202,1000,7890,22479,131294,22934,85714,85714,79841,5873,4714,1758,1084,32,642,0,2956,1450,1321,185,47,125
"Rutherford County, Tennessee",0500000US47149,341486,341486,226227,53977,1739,12538,254,19755,26996,219194,38317,131216,131216,124465,6751,5723,1851,774,43,955,79,3872,2831,0,1041,47,149
"Sumner County, Tennessee",0500000US47165,196281,196281,158075,15740,711,2956,119,5321,13359,155169,12870,78995,78995,74472,4523,1612,1377,572,25,780,0,235,0,0,235,47,165
"Trousdale County, Tennessee",0500000US47169,11615,11615,8883,1922,30,27,0,205,548,8786,375,3749,3749,3469,280,2569,2549,2505,0,44,0,20,0,0,20,47,169
"Williamson County, Tennessee",0500000US47187,247726,247726,203600,9876,574,12953,117,5172,15434,200408,14261,91133,91133,86884,4249,1359,1252,189,28,1035,0,107,0,0,107,47,187


In [12]:
transp = data.transpose()
transp.head()

NAME,"Montgomery County, Tennessee","Rutherford County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee","Cheatham County, Tennessee","Macon County, Tennessee","Robertson County, Tennessee","Stewart County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Simpson County, Kentucky","Allen County, Kentucky","Adams city, Tennessee","Ashland City town, Tennessee","Belle Meade city, Tennessee","Berry Hill city, Tennessee","Brentwood city, Tennessee","Burns town, Tennessee","Cedar Hill city, Tennessee","Charlotte town, Tennessee","Clarksville city, Tennessee","Columbia city, Tennessee","Coopertown town, Tennessee","Cross Plains city, Tennessee","Cumberland City town, Tennessee","Dickson city, Tennessee","Dover city, Tennessee","Eagleville city, Tennessee","Erin city, Tennessee","Fairview city, Tennessee","Forest Hills city, Tennessee","Franklin city, Tennessee","Gallatin city, Tennessee","Goodlettsville city, Tennessee","Greenbrier town, Tennessee","Hendersonville city, Tennessee","Kingston Springs town, Tennessee","Lafayette city, Tennessee","La Vergne city, Tennessee","Lebanon city, Tennessee","McEwen city, Tennessee","Millersville city, Tennessee","Mitchellville city, Tennessee","Mount Juliet city, Tennessee","Mount Pleasant city, Tennessee","Murfreesboro city, Tennessee","Nashville-Davidson metropolitan government (balance), Tennessee","New Johnsonville city, Tennessee","Nolensville town, Tennessee","Oak Hill city, Tennessee","Pegram town, Tennessee","Pleasant View city, Tennessee","Portland city, Tennessee","Ridgetop city, Tennessee","Slayden town, Tennessee","Smyrna town, Tennessee","Springfield city, Tennessee","Spring Hill city, Tennessee","Tennessee Ridge town, Tennessee","Thompson's Station town, Tennessee","Vanleer town, Tennessee","Watertown city, Tennessee","Waverly city, Tennessee","Westmoreland town, Tennessee","White Bluff town, Tennessee","White House city, Tennessee","Scottsville city, Kentucky","Franklin city, Kentucky",Tennessee,United States
GEO_ID,0500000US47125,0500000US47149,0500000US47165,0500000US47169,0500000US47187,0500000US47189,0500000US47021,0500000US47111,0500000US47147,0500000US47161,0500000US47037,0500000US47043,0500000US47083,0500000US47085,0500000US47119,0500000US21213,0500000US21003,1600000US4700200,1600000US4702180,1600000US4704620,1600000US4705140,1600000US4708280,1600000US4709880,1600000US4711980,1600000US4713080,1600000US4715160,1600000US4716540,1600000US4716980,1600000US4718420,1600000US4718820,1600000US4720620,1600000US4721400,1600000US4722360,1600000US4724320,1600000US4725440,1600000US4727020,1600000US4727740,1600000US4728540,1600000US4729920,1600000US4730960,1600000US4733280,1600000US4739660,1600000US4740160,1600000US4741200,1600000US4741520,1600000US4744840,1600000US4748980,1600000US4749460,1600000US4750780,1600000US4751080,1600000US4751560,1600000US4752006,1600000US4752820,1600000US4753460,1600000US4754780,1600000US4757480,1600000US4759560,1600000US4760280,1600000US4763140,1600000US4769080,1600000US4769420,1600000US4770500,1600000US4770580,1600000US4773460,1600000US4773900,1600000US4776860,1600000US4778320,1600000US4778560,1600000US4779420,1600000US4779980,1600000US4780200,1600000US2169114,1600000US2128918,0400000US47,0100000US
pop,220069,341486,196281,11615,247726,147737,41072,25216,72803,13657,715884,54315,8283,18990,100974,19594,20588,624,5193,2901,2112,45373,1573,301,1656,166722,41690,4480,1789,305,16058,1826,813,1224,9357,5038,83454,44431,17789,6898,61753,2824,5584,38719,38431,1643,6299,163,39289,4784,152769,689447,1804,13829,4891,2072,4807,13156,2155,170,53070,18782,50005,1332,7485,374,1553,4297,2718,3862,12982,4299,10176,6910840,331449281
raceeth_total_series,220069,341486,196281,11615,247726,147737,41072,25216,72803,13657,715884,54315,8283,18990,100974,19594,20588,624,5193,2901,2112,45373,1573,301,165

In [13]:
numcols = list(transp.columns)
numcols
transp[numcols] = transp[numcols].astype(float)

In [14]:
data = transp

In [15]:
GNRCCounties = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                data[('Rutherford County, Tennessee', '0500000US47149')]]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                   data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                   data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                   data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                   data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                   data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                   data[('Rutherford County, Tennessee', '0500000US47149')],data[('Maury County, Tennessee', '0500000US47119')]]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
               data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
               data[('Williamson County, Tennessee', '0500000US47187')],data[('Rutherford County, Tennessee', '0500000US47149')],
               data[('Maury County, Tennessee', '0500000US47119')]]
data['MPO'] = sum(MPOCounties)
RuthInc = [data[('Eagleville city, Tennessee', '1600000US4722360')],data[('La Vergne city, Tennessee', '1600000US4741200')],
           data[('Murfreesboro city, Tennessee', '1600000US4751560')],data[('Smyrna town, Tennessee', '1600000US4769420')]]
data[('Rutherford Incorporated', 'None')] = sum(RuthInc)
data[('Rutherford Unincorporated', 'None')] = data[('Rutherford County, Tennessee', '0500000US47149')] - data[('Rutherford Incorporated', 'None')]
WilsonInc = [data[('Lebanon city, Tennessee', '1600000US4741520')],data[('Mount Juliet city, Tennessee', '1600000US4750780')],
             data[('Watertown city, Tennessee', '1600000US4778320')]]
data[('Wilson Incorporated', 'None')] = sum(WilsonInc)
data[('Wilson Unincorporated', 'None')] = data[('Wilson County, Tennessee', '0500000US47189')] - data[('Wilson Incorporated', 'None')]
CheathInc = [data[('Ashland City town, Tennessee', '1600000US4702180')],data[('Kingston Springs town, Tennessee', '1600000US4739660')],
             data[('Pegram town, Tennessee', '1600000US4757480')],data[('Pleasant View city, Tennessee', '1600000US4759560')]]
data[('Cheatham Incorporated', 'None')] = sum(CheathInc)
data[('Cheatham Unincorporated', 'None')] = data[('Cheatham County, Tennessee', '0500000US47021')] - data[('Cheatham Incorporated', 'None')]
DicksInc = [data[('Burns town, Tennessee', '1600000US4709880')],data[('Charlotte town, Tennessee', '1600000US4713080')],
            data[('Dickson city, Tennessee', '1600000US4720620')],data[('Slayden town, Tennessee', '1600000US4769080')],
            data[('Vanleer town, Tennessee', '1600000US4776860')],data[('White Bluff town, Tennessee', '1600000US4779980')]]
data[('Dickson Incorporated', 'None')] = sum(DicksInc)
data[('Dickson Unincorporated', 'None')] = data[('Dickson County, Tennessee', '0500000US47043')] - data[('Dickson Incorporated', 'None')]
HumphInc = [data[('McEwen city, Tennessee', '1600000US4744840')],data[('New Johnsonville city, Tennessee', '1600000US4752820')],
            data[('Waverly city, Tennessee', '1600000US4778560')]]
data[('Humphreys Incorporated', 'None')] = sum(HumphInc)
data[('Humphreys Unincorporated', 'None')] = data[('Humphreys County, Tennessee', '0500000US47085')] - data[('Humphreys Incorporated', 'None')]
data[('Montgomery Incorporated', 'None')] = data[('Clarksville city, Tennessee', '1600000US4715160')]
data[('Montgomery Unincorporated', 'None')] = data[('Montgomery County, Tennessee', '0500000US47125')] - data[('Montgomery Incorporated', 'None')]

In [16]:
data = data.transpose().reset_index()

In [17]:
data.head()

,NAME,GEO_ID,pop,raceeth_total_series,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,gq_total_series,gq_institutionalized,gq_institutionalized_correctionalfacilityadults,gq_institutionalized_juvenilefacility,gq_institutionalized_nursingorskillednursingfacility,gq_institutionalized_other,gq_noninstitutionalized,gq_noninstitutionalized_collegeoruniversitystudenthousing,gq_noninstitutionalized_militaryquarters,gq_noninstitutionalized_other,StateFIPS,GeoFIPS
0,"Montgomery County, Tennessee",0500000US47125,220069.0,220069.0,137775.0,44569.0,1154.0,5202.0,1000.0,7890.0,22479.0,131294.0,22934.0,85714.0,85714.0,79841.0,5873.0,4714.0,1758.0,1084.0,32.0,642.0,0.0,2956.0,1450.0,1321.0,185.0,47.0,125.0
1,"Rutherford County, Tennessee",0500000US47149,341486.0,341486.0,226227.0,53977.0,1739.0,12538.0,254.0,19755.0,26996.0,219194.0,38317.0,131216.0,131216.0,124465.0,6751.0,5723.0,1851.0,774.0,43.0,955.0,79.0,3872.0,2831.0,0.0,1041.0,47.0,149.0
2,"Sumner County, Tennessee",0500000US47165,196281.0,196281.0,158075.0,15740.0,711.0,2956.0,119.0,5321.0,13359.0,155169.0,12870.0,78995.0,78995.0,74472.0,4523.0,1612.0,1377.0,572.0,25.0,780.0,0.0,235.0,0.0,0.0,235.0,47.0,165.0
3,"Trousdale County, Tennessee",0500000US47169,11615.0,11615.0,8883.0,1922.0,30.0,27.0,0.0,205.0,548.0,8786.0,375.0,3749.0,3749.0,3469.0,280.0,2569.0,2549.0,2505.0,0.0,44.0,0.0,20.0,0.0,0.0,20.0,47.0,169.0
4,"Williamson County, Tennessee",0500000US47187,247726.0,247726.0,203600.0,9876.0,574.0,12953.0,117.0,5172.0,15434.0,200408.0,14261.0,91133.0,91133.0,86884.0,4249.0,1359.0,1252.0,189.0,28.0,1035.0,0.0,107.0,0.0,0.0,107.0,47.0,187.0


In [19]:
data.to_csv('../../Data/PL2020.csv', index = False)